In [1]:
from sklearn.model_selection import GridSearchCV
import numpy as np

X = np.load('data/X_train.npy')#[0:load_n]
y = np.genfromtxt('data/y_1.csv', delimiter='\n')#[0:load_n]
#data_test = np.load('data/X_test.npy')


In [12]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.base import BaseEstimator, TransformerMixin
import random
random.seed(99)

class GivenBinHistExtraction(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_new = []
        X_new = X
        return X_new

class ClusteredHistExtraction(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, n_samples=3, images_x_from=False, images_x_to=False):
        self.n_clusters = n_clusters
        self.n_samples = n_samples
        self.images_x_from = images_x_from
        self.images_x_to = images_x_to


    def cutImage(self, x):
        if self.images_x_from is not False and self.images_x_to is not False:
            #images = np.split(row, 176)[50:130] # pretty optimal already
            side_images = np.split(x, 176)[self.images_x_from : self.images_x_to]
            x = np.array(side_images).flatten()
        return x


    def fit(self, X, y=None):
        samples = random.sample(list(X), self.n_samples)
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_jobs=-1, random_state=42)
        # self.kmeans = MiniBatchKMeans(n_clusters=self.n_clusters, batch_size=100, random_state=42)

        centers = []
        for i, sample in enumerate(samples):
            # sample = sample[1672390 : -786303]
            sample = self.cutImage(sample)

            sample = sample[(sample > 0) & (sample < 1800)]
            self.kmeans.fit(np.array([sample]).T)
            centers.append(np.sort(np.array(self.kmeans.cluster_centers_).flatten()))
            
            samples[i] = sample
            #print(str(i) + ' done')

        if True: # use all centers
            values = np.array(centers).flatten()
            values = np.sort(values)
        else: # take means of centers
            values = np.mean(centers, axis=0)

        # compute cluster centers
        #self.kmeans.fit(np.array(samples).T)
        #values = self.kmeans.cluster_centers_.T
        #print('fitted')

        # mean of the clusters over the rows
        #for i, v in enumerate(values.T):
        #    values.T[i] = np.sort(v)

        #values = np.mean(values.T, axis=0)

        self.edges = [1] # leave out 0
        for center_1, center_2 in zip(values[:-1], values[1:]):
            self.edges.append(.5 * (center_1 + center_2))

        print('n edges: ' + str(len(self.edges)))
        return self

    def transform(self, X, y=None):
        # np.histogram to make bins from edges, counts the number of pixels
        X_new = []
        for x in X:
            x = self.cutImage(x)
            x = x[(x > 0) & (x < 1800)]
            hist = np.histogram(x, bins=self.edges)
            X_new.append(hist[0])

        return X_new
    
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.kernel_ridge import KernelRidge

pipe = Pipeline([
    ('ClusteredHistExtraction', ClusteredHistExtraction(
        n_clusters=8,n_samples=3)),
    ('scaler', StandardScaler()),
    ('vct', VarianceThreshold()),
    ('kernelRidge', KernelRidge(kernel='polynomial'))
])

In [13]:
from sklearn.model_selection import GridSearchCV

parameters = {
 #   'ClusteredHistExtraction__n_clusters': [8, 10],
 #   'ClusteredHistExtraction__n_samples': [3, 4]
}
grid = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(str(np.mean(grid.cv_results_['mean_test_score'])) + ', ' + str(np.mean(grid.cv_results_['std_test_score'])))

# -68.8529277579, 5.72349220142
# -68.5395679288, 6.39975328753

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done
0 done
1 done
1 done
1 done
1 done
2 done
n edges: 24
2 done
n edges: 24
2 done
n edges: 24
2 done
3 done
n edges: 32


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1 done
1 done
1 done
2 done
0 done
2 done
3 done
n edges: 32
2 done
n edges: 30
3 done
n edges: 32
1 done
2 done
n edges: 30


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


0 done
0 done
0 done


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


1 done
1 done
1 done
0 done
2 done
2 done
n edges: 30
2 done
1 done
3 done
n edges: 40
3 done
n edges: 40
2 done
3 done
n edges: 40
0 done
1 done
2 done
n edges: 24
-68.5395679288, 6.39975328753


In [17]:
grid.cv_results_
#grid.best_params_

{'ClusteredHistExtraction__n_clusters': 8,
 'ClusteredHistExtraction__n_samples': 3}

In [20]:
# Search bin subdivision so that there is highest variance
# 1. Divide hist into bins over some rows
# 2. Compute std -> divide bins where it's high, join/drop those where it's low
# 3. Repeat / change rows
kmeans = KMeans(n_clusters=8, n_jobs=-1, random_state=42)

samples = X[0: 5]
centers = []
for i, sample in enumerate(samples):
    # sample = sample[1672390 : -786303]

    sample = sample[(sample > 0) & (sample < 1800)]
    kmeans.fit(np.array([sample]).T)
    centers.append(np.sort(np.array(kmeans.cluster_centers_).flatten()))

    #samples[i] = sample
    #print(str(i) + ' done')

if True: # use all centers
    values = np.array(centers).flatten()
    values = np.sort(values)
else: # take means of centers
    values = np.mean(centers, axis=0)

edges = [1] # leave out 0
for center_1, center_2 in zip(values[:-1], values[1:]):
    edges.append(.5 * (center_1 + center_2))

print('n edges: ' + str(len(edges)))
edges

n edges: 40


[1,
 268.59153353677925,
 281.00824805433393,
 296.51672876785585,
 312.41929109857142,
 385.23831822211014,
 458.12983660320458,
 472.33076634277745,
 493.82267048266544,
 510.43647791541645,
 576.57399025095651,
 642.540938664245,
 648.54789979548752,
 655.08349269468147,
 666.54241844275282,
 724.33201508885827,
 779.89739389945362,
 783.20843221048995,
 785.82438478516588,
 807.58109023044335,
 869.72060524581752,
 913.23145834367801,
 915.91949110352323,
 919.63925603546056,
 950.7720301599602,
 1021.6951866283912,
 1069.5507893466106,
 1076.2284155454829,
 1078.4194846165499,
 1107.9840749772698,
 1178.3353557602009,
 1228.4902326692686,
 1237.3030234970834,
 1243.0805123468485,
 1277.8951157600625,
 1339.3753602258789,
 1372.8570616285097,
 1379.1746465921135,
 1388.0591113102987,
 1433.3192436369491]

In [54]:
from sklearn.utils import resample

edges = [1,
 200,
 225,
 240,
 260.59153353677925,
 281.00824805433393,
 296.51672876785585,
 312.41929109857142,
 385.23831822211014,
 458.12983660320458,
 493.82267048266544,
 510.43647791541645,
 576.57399025095651,
 642.540938664245,
 655.08349269468147,
 666.54241844275282,
 724.33201508885827,
 779.89739389945362,
 783.20843221048995,
 785.82438478516588,
 807.58109023044335,
 869.72060524581752,
 913.23145834367801,
 919.63925603546056,
 950.7720301599602,
 1021.6951866283912,
 1069.5507893466106,
 1178.3353557602009,
 1228.4902326692686,
 1277.8951157600625,
 1339.3753602258789,
 1372.8570616285097,
 1433.3192436369491,
 1500,
 1600]
hists = []
for x in resample(X, n_samples=40):
    hists.append(np.histogram(x, bins=edges)[0])
    
std_all = np.std(np.array(hists).flatten())
for bin_i in range(len(hists[0])):
    arr = [h[bin_i] for h in hists]
    print("bin "+str(bin_i)+": "+str(np.std(arr) / std_all))
#edges

bin 0: 0.239414873916
bin 1: 0.298692302763
bin 2: 0.20953146529
bin 3: 0.273875225584
bin 4: 0.229888507051
bin 5: 0.136261113227
bin 6: 0.126734212997
bin 7: 0.438634532093
bin 8: 0.316206196958
bin 9: 0.117589064067
bin 10: 0.0490891973573
bin 11: 0.136280655
bin 12: 0.25035747048
bin 13: 0.0940809990061
bin 14: 0.0943191582988
bin 15: 0.719163577413
bin 16: 0.864481550987
bin 17: 0.0589738384951
bin 18: 0.0294721829112
bin 19: 0.297933166334
bin 20: 0.55437517554
bin 21: 0.236974125554
bin 22: 0.0324723392439
bin 23: 0.163588130985
bin 24: 0.358460322697
bin 25: 0.190580795313
bin 26: 0.293258584745
bin 27: 0.215387024907
bin 28: 0.370939674545
bin 29: 0.780736952028
bin 30: 0.370734128914
bin 31: 0.629354214444
bin 32: 0.88052005338
bin 33: 0.82044039959
